The source code is in the public domain and not licensed or under
copyright. The information and software may be used freely by the public.
As required by 17 U.S.C. 403, third parties producing copyrighted works
consisting predominantly of the material produced by U.S. government
agencies must provide notice with such work(s) identifying the U.S.
Government material incorporated and stating that such material is not
subject to copyright protection.

Derived works shall not identify themselves in a manner that implies an
endorsement by or an affiliation with the Naval Research Laboratory.

RECIPIENT BEARS ALL RISK RELATING TO QUALITY AND PERFORMANCE OF THE
SOFTWARE AND ANY RELATED MATERIALS, AND AGREES TO INDEMNIFY THE NAVAL
RESEARCH LABORATORY FOR ALL THIRD-PARTY CLAIMS RESULTING FROM THE ACTIONS
OF RECIPIENT IN THE USE OF THE SOFTWARE.

In [3]:
import os
import sys
import glob
import warnings
import json
import pathlib
import numpy as np
from scipy import constants as C
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets
import IPython.display
import twutils.plot as twplot
%matplotlib widget
control_layout = 'left' # can be 'left' or 'bottom'; at present bottom is clunky but works
image_height_px = 600 # width is based on window size assuming css styles are correctly installed

# Select buffering mode: True reads the entire array at once, False uses memory mapping
buffered_mode = True

mpl.rcParams.update({'font.size' : 12})

# Starting values

undefine_list = ['ax','fig','lineout','image','bar','plotter','dims','init_fig','transform_chain']
for gv in undefine_list:
    if gv in globals():
        del globals()[gv]

favorite_file = 'Ex.npy'
favorite_color = 'jet'
favorite_units = 'plasma'
start_path = pathlib.Path.home() / 'Run'
file_list = []
color_list = ['viridis','magma','plasma','inferno',
            'Spectral','bwr','seismic','prism','flag','ocean',
            'rainbow','jet','nipy_spectral','gist_ncar','gist_earth']
units_list = ['mks','cgs','plasma','atomic','natural']
# The following must be in the given form (they are more than labels)
plane_list = ['xytz','zxty','zytx','txyz','tyxz','tzxy']
disp_list = ['Falsecolor-2D','Horizontal-Lineout','Vertical-Lineout']
transform_list = ['abs(x)','Log_e(abs(x))','Log_10(abs(x))','x^2','-x']
transform_chain = []

fig_pane = ipywidgets.Output(layout=ipywidgets.Layout(width='100%'))
param_w = ipywidgets.Textarea(value='no data',description='Metrics')
path_w = ipywidgets.Text(value=str(start_path),description='Path',disabled=True)
file_w = ipywidgets.Dropdown(options=['no files'],value='no files',description='File')
plane_w = ipywidgets.Dropdown(options=plane_list,value='zxty',description='Plane')
disp_w = ipywidgets.Dropdown(options=disp_list,value=disp_list[0],description='Display')
frame_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,description='Frame',continuous_update=False)
slice_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,description='Slice',continuous_update=False)
hcurs_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,description='H-Cursor',continous_update=False)
vcurs_w = ipywidgets.IntSlider(min=0,max=0,step=1,value=0,description='V-Cursor',continous_update=False)
lim1_w = ipywidgets.FloatText(value=-1.0,description='Min')
lim2_w = ipywidgets.FloatText(value=1.0,description='Max')
color_w = ipywidgets.Dropdown(options=color_list,value=favorite_color,description='Colors')
units_w = ipywidgets.Dropdown(options=units_list,value=favorite_units,description='Units')
inv_color_w = ipywidgets.Checkbox(value=False,description='Invert Colors')
prop_w = ipywidgets.Checkbox(value=False,description='Proportional')
tex_w = ipywidgets.Checkbox(value=False,description='Use TeX')
dpi_w = ipywidgets.FloatText(value=100,description='DPI')
autoscale_frame_w = ipywidgets.Button(description='Autoscale Frame')
autoscale_all_w = ipywidgets.Button(description='Autoscale All')
transform_select_w = ipywidgets.Dropdown(options=transform_list,value=transform_list[0],layout=ipywidgets.Layout(width='100px'))
transform_w = ipywidgets.Button(description='Transform')

def setup_files(file_path):
    global init_fig
    init_fig = True
    # Get the file list in the specified path
    os.chdir(file_path)
    try:
        with open('tw_metadata.json','r') as f:
            meta = json.load(f)
    except (FileNotFoundError,PermissionError):
        warnings.warn('Could not read metadata file ('+str(sys.exc_info()[0])+').')
        meta = {}
    file_list = glob.glob('*.npy')
    file_list.sort()
    # Exclude files not in the metadata dictionary
    filtered = []
    for f in file_list:
        if f in meta:
            filtered += [f]
    if len(filtered)==0:
        filtered = file_list
    file_w.options = filtered
    if favorite_file in filtered:
        file_w.value = favorite_file
    else:
        file_w.value = filtered[0]
    
def switch_axes_primitive():
    axis_lab_map = { 0 : 't' , 1 : 'x' , 2 : 'y' , 3 : 'z' }
    axi = twplot.get_axis_info(plane_w.value)
    frame_w.description = 'Frame (' + axis_lab_map[axi[2]] + ')'
    frame_w.max = dims[axi[2]]-1
    frame_w.value = 0
    slice_w.description = 'Slice (' + axis_lab_map[axi[3]] + ')'
    slice_w.max = dims[axi[3]]-1
    slice_w.value = dims[axi[3]]/2
    hcurs_w.max = dims[axi[0]]-1
    hcurs_w.value = dims[axi[0]]/2
    vcurs_w.max = dims[axi[1]]-1
    vcurs_w.value = dims[axi[1]]/2

def switch_axes(plane_spec):
    global init_fig
    init_fig = True
    switch_axes_primitive()

def setup_plots(file_to_plot,units):
    global plotter,dims,init_fig,transform_chain
    init_fig = True
    transform_chain = []
    theFile = str(pathlib.Path(path_w.value) / file_to_plot)
    plotter = twplot.plotter(theFile,units,buffered=buffered_mode)
    dims = plotter.dims4()
    param_w.value = '(t,x,y,z) = '+str(dims)+'\n'
    switch_axes_primitive()
    auto_scale_all(autoscale_all_w)
    disp_frame(frame_w.value,slice_w.value,hcurs_w.value,vcurs_w.value)

def auto_scale_frame(the_button):
    axi = twplot.get_axis_info(plane_w.value)
    # Extract the frame slice
    frame_slice = np.take(plotter.get(),[frame_w.value],axis=axi[2])
    # Extract the "slice slice"
    data_slice = np.take(frame_slice,[slice_w.value],axis=axi[3])
    lim1_w.value = np.min(data_slice)
    lim2_w.value = np.max(data_slice)
    
def auto_scale_all(the_button):
    global init_fig
    init_fig = True
    lim1_w.value = np.min(plotter.get())
    lim2_w.value = np.max(plotter.get())

def transform_data(the_button):
    global transform_chain
    smallest_num = 1e-25
    if transform_select_w.value=='Log_e(abs(x))':
        plotter.set(np.log(smallest_num+np.abs(plotter.get())))
        transform_chain += ['abs',r'log$_e$']
    if transform_select_w.value=='Log_10(abs(x))':
        plotter.set(np.log10(smallest_num+np.abs(plotter.get())))
        transform_chain += ['abs',r'log$_{10}$']
    if transform_select_w.value=='x^2':
        plotter.set(plotter.get()**2)
        transform_chain += ['sqr']
    if transform_select_w.value=='-x':
        plotter.scale(-1.0)
        transform_chain += ['(-)']
    if transform_select_w.value=='abs(x)':
        plotter.set(np.abs(plotter.get()))
        transform_chain += ['abs']
    auto_scale_all(autoscale_all_w)
    
def annotate_label(label,chain):
    lab = label
    for T in chain:
        lab = T + lab
    while 'abs' in lab:
        lab = lab.replace('abs',r'$|$') + r'$|$'
    return lab

def force_update(lim1,lim2,color_map,inv_color,prop,tex,dpi,disp):
    global init_fig
    init_fig = True
    disp_frame(frame_w.value,slice_w.value,hcurs_w.value,vcurs_w.value)

def disp_frame(f,s,h,v):
    global init_fig
    global plotter, ax, fig, lineout, image, bar
    if init_fig:
        if inv_color_w.value:
            color_map = color_w.value + '_r'
        else:
            color_map = color_w.value
        if prop_w.value:
            my_aspect = 'equal'
        else:
            my_aspect = 'auto'
        fig_pane.clear_output(wait=True)
        mpl.rcParams['text.usetex'] = tex_w.value
        plt.close('all')
        with fig_pane:
            fig,ax = plt.subplots(constrained_layout=True,dpi=dpi_w.value)
            fig.canvas.layout.height = str(image_height_px)+'px'
            fig.canvas.layout.flex = '1 1 auto'
    slicing = plane_w.value
    if disp_w.value=='Falsecolor-2D':
        data_slice,plot_dict = plotter.falsecolor2d(slicing,(f,s))
        hc = plotter.plot_coord(plot_dict,0,h)
        vc = plotter.plot_coord(plot_dict,1,v) 
        if init_fig:
            image = ax.imshow(data_slice,vmin=lim1_w.value,vmax=lim2_w.value,origin='lower',aspect=my_aspect,extent=plot_dict['extent'],cmap=color_map)
            lineout, = ax.plot([hc],[vc],'ro')
            bar = fig.colorbar(image,ax=ax)
            lab = annotate_label(plot_dict['blabel'],transform_chain)
            ax.set_xlabel(plot_dict['xlabel'],fontsize=12)
            ax.set_ylabel(plot_dict['ylabel'],fontsize=12)
            bar.set_label(lab)
        else:
            image.set_data(data_slice)
            lineout.set_xdata(hc)
            lineout.set_ydata(vc)
    if disp_w.value[-7:]=='Lineout':
        if disp_w.value[:3]=='Hor':
            abcissa,ordinate,plot_dict = plotter.lineout(slicing,(v,f,s))
        else:
            vslicing = slicing[1]+slicing[0]+slicing[2]+slicing[3]
            abcissa,ordinate,plot_dict = plotter.lineout(vslicing,(h,f,s))
        if init_fig:
            lineout, = ax.plot(abcissa,ordinate)
            plt.ylim(lim1_w.value,lim2_w.value)
            lab = plot_dict['ylabel']
            for T in transform_chain:
                lab = T + lab
            ax.set_xlabel(plot_dict['xlabel'],fontsize=12)
            ax.set_ylabel(lab,fontsize=12)
        else:
            lineout.set_ydata(ordinate)
    init_fig = False

# Initialize the plotter object and plot widget
# Related interactive widgets have to be created first

setup_iw = ipywidgets.interactive(setup_plots,file_to_plot = file_w,units = units_w)
advance_iw = ipywidgets.interactive(disp_frame,f=frame_w,s=slice_w,h=hcurs_w,v=vcurs_w)
setup_files(path_w.value)
setup_plots(file_w.value,units_w.value)
disp_frame(frame_w.value,slice_w.value,hcurs_w.value,vcurs_w.value)

# Set up layouts

autoscale_box = ipywidgets.HBox([autoscale_frame_w,autoscale_all_w])
transform_box = ipywidgets.HBox([transform_select_w,transform_w])
left_box = ipywidgets.VBox([frame_w,slice_w,hcurs_w,vcurs_w,param_w,path_w,file_w,lim1_w,lim2_w,units_w,color_w,inv_color_w,
                            prop_w,tex_w,dpi_w,plane_w,disp_w,autoscale_box,transform_box],
                            layout=ipywidgets.Layout(width='330px',flex='0 0 auto'))

if control_layout=='left':
    main_view = ipywidgets.HBox([left_box, fig_pane],
                            layout=ipywidgets.Layout(width='100%',height=str(image_height_px+100)+'px',display='flex',
                                                     align_items='stretch',
                                                     align_content='flex-start',
                                                     justify_content='flex-start'))
else:
    main_view = ipywidgets.VBox([fig_pane, left_box])

# Set up interactive elements

update_iw = ipywidgets.interactive(force_update,lim1=lim1_w,lim2=lim2_w,color_map=color_w,inv_color=inv_color_w,prop=prop_w,tex=tex_w,dpi=dpi_w,disp=disp_w)
switch_iw = ipywidgets.interactive(switch_axes,plane_spec=plane_w)

autoscale_frame_w.on_click(auto_scale_frame)
autoscale_all_w.on_click(auto_scale_all)
transform_w.on_click(transform_data)

IPython.display.display(main_view)